In [1]:
import datetime
import pickle

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR

In [2]:
configs = {
    'batch_size': 16,
    'epochs': 300,
#     'epochs': 2,  # for test
    'learning_rate': 1e-3,
    'weight_decay': 1e-6,
    'es_patience': 10,
}

time_now = (datetime.datetime.now()+datetime.timedelta(hours=9)).strftime("%Y-%m-%d %H:%M:%S")
configs['time_now'] = time_now

In [3]:
with open('../data/train_prob1.pkl', 'rb') as f:
    train_origin = pickle.load(f)
with open('../data/test_prob1.pkl', 'rb') as f:
    test_origin = pickle.load(f)

In [4]:
train, test = train_origin.copy(), test_origin.copy()

train = train[:50000]

train['status'] = train['status'].apply(lambda x: int(x[-1]))
test['status'] = test['status'].apply(lambda x: int(x[-1]))

train, valid = train_test_split(train)
train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)

In [5]:
input_cols = ['distance', 'fare', 'call_count', 'dispatch_count', 'driver_count']
target_col = 'status'

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device {device}')
configs['device'] = device

Using device cuda


In [7]:
class TaxiDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        inputs = self.data.iloc[idx, :-1]
        inputs = torch.tensor(inputs).type(torch.float)
        targets = self.data.iloc[idx, -1]
        targets = torch.tensor(targets).type(torch.long)
        return inputs, targets

In [8]:
train_to_input = train[input_cols + [target_col]]
valid_to_input = valid[input_cols + [target_col]]
test_to_input = test[input_cols + [target_col]]

train_ds = TaxiDataset(train_to_input)
valid_ds = TaxiDataset(valid_to_input)
test_ds = TaxiDataset(test_to_input)

train_dl = DataLoader(train_ds, batch_size=configs['batch_size'], shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=configs['batch_size'])
test_dl = DataLoader(test_ds, batch_size=configs['batch_size'])

In [9]:
class NNModel(nn.Module):
    def __init__(self):
        super(NNModel, self).__init__()
        self.dense_relu_stack = nn.Sequential(
            nn.Linear(len(input_cols), 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 3),
        )
        
    def forward(self, inputs):
        outputs = self.dense_relu_stack(inputs)
        return outputs
    
model = NNModel().to(device)
print(model)

NNModel(
  (dense_relu_stack): Sequential(
    (0): Linear(in_features=5, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=3, bias=True)
  )
)


In [10]:
loss_fn = nn.CrossEntropyLoss(reduction='sum')
optimizer = Adam(model.parameters(), lr=configs['learning_rate'])
lr_scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=configs['learning_rate']/20)

In [11]:
def train_one_epoch(train_dl, model, loss_fn, optimizer, lr_scheduler, configs):
    
    n_total = len(train_dl.dataset)
    
    train_loss = 0
    train_acc = 0
    
    for batch, (X, y) in enumerate(train_dl):
        X, y = X.to(configs['device']), y.to(configs['device'])
        
        pred = model(X)
        loss = loss_fn(pred, y)
        acc = (pred.argmax(axis=1) == y).sum().item()
        
        train_loss += loss.item()/n_total
        train_acc += acc/n_total
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        
        check = (n_total//configs['batch_size']+1)//5
        if batch%check == check-1:
            train_loss_tmp = train_loss*n_total/(batch+1)/configs['batch_size']
            train_acc_tmp = train_acc*n_total/(batch+1)/configs['batch_size']
            current = batch*configs['batch_size']
            print(f'loss: {train_loss_tmp:>10f}, acc: {train_acc_tmp*100:>10f} [{current:>5d}/{n_total:>5d}]')
            
    return train_loss, train_acc

In [12]:
def evaluate(dl, model, loss_fn, configs, return_pred=False):
    
    n_total = len(dl.dataset)
    
    total_loss = 0
    total_acc = 0
    
    if return_pred:
        pred_arr = []
    
    model.eval()
    with torch.no_grad():
        for X, y in dl:
            X, y = X.to(configs['device']), y.to(configs['device'])

            pred = model(X)
            if return_pred:
                pred_arr.extend(pred.tolist())
            loss = loss_fn(pred, y)
            acc = (pred.argmax(axis=1) == y).sum().item()

            total_loss += loss.item()/n_total
            total_acc += acc/n_total
            
    if return_pred:
        return total_loss, total_acc, np.array(pred_arr)
    else:
        return total_loss, total_acc

In [13]:
best_valid_loss = np.inf
best_valid_acc = 0

early_stop = 0
for epoch in range(configs['epochs']):
    lr_now = optimizer.param_groups[0]['lr']
    print(f'\nEpoch {epoch+1}\nLearning Rate {lr_now:>8f}\n---------------------------------')
    train_loss, train_acc = train_one_epoch(train_dl, model, loss_fn, optimizer, lr_scheduler, configs)
    valid_loss, valid_acc = evaluate(valid_dl, model, loss_fn, configs)
    print(f'\nTrain Loss: {train_loss:>8f}, Train Acc: {train_acc*100:>3.3f}%')
    print(f'Valid Loss: {valid_loss:>8f}, Valid Acc: {valid_acc*100:>3.3f}%\n')
    
    if best_valid_loss > valid_loss:
        print(f'Break the best valid loss\n\t{best_valid_loss:>8f} -> {valid_loss:>8f}\n')
        best_train_loss = train_loss
        best_train_acc = train_acc
        best_valid_loss = valid_loss
        best_valid_acc = valid_acc
        
        torch.save(model.state_dict(), '../model/ka_s_model.pth')
        
        early_stop = 0
    else:
        early_stop += 1
        
    if early_stop >= configs['es_patience']:
        break
        
print(f'Best Model')
print(f'\tbest_train_loss: {best_train_loss}, best_train_acc: {best_train_acc}')
print(f'\tbest_valid_loss: {best_valid_loss}, best_valid_acc: {best_valid_acc}')


Epoch 1
Learning Rate 0.001000
---------------------------------
loss: 171.858952, acc:  40.918803 [ 7472/37500]
loss:  89.953350, acc:  42.327724 [14960/37500]
loss:  62.344645, acc:  42.997685 [22448/37500]
loss:  48.365558, acc:  43.352698 [29936/37500]
loss:  39.616196, acc:  43.955662 [37424/37500]

Train Loss: 39.565042, Train Acc: 43.952%
Valid Loss: 2.802052, Valid Acc: 50.480%

Break the best valid loss
	     inf -> 2.802052


Epoch 2
Learning Rate 0.000672
---------------------------------
loss:   6.602284, acc:  43.643162 [ 7472/37500]
loss:   7.056515, acc:  44.477831 [14960/37500]
loss:   6.960096, acc:  44.689281 [22448/37500]
loss:   6.770092, acc:  44.858440 [29936/37500]
loss:   6.548911, acc:  44.863782 [37424/37500]

Train Loss: 6.545806, Train Acc: 44.845%
Valid Loss: 3.417785, Valid Acc: 50.552%


Epoch 3
Learning Rate 0.000141
---------------------------------
loss:   5.428139, acc:  44.537927 [ 7472/37500]
loss:   4.898378, acc:  45.252404 [14960/37500]
loss:   

loss:   0.925560, acc:  44.885150 [ 7472/37500]
loss:   0.934887, acc:  45.032051 [14960/37500]
loss:   0.932169, acc:  45.263533 [22448/37500]
loss:   0.933435, acc:  45.332532 [29936/37500]
loss:   0.933382, acc:  45.422009 [37424/37500]

Train Loss: 0.933579, Train Acc: 45.432%
Valid Loss: 0.929522, Valid Acc: 44.864%


Epoch 22
Learning Rate 0.000672
---------------------------------
loss:   0.937066, acc:  45.312500 [ 7472/37500]
loss:   0.931503, acc:  45.779915 [14960/37500]
loss:   0.931571, acc:  45.637464 [22448/37500]
loss:   0.931970, acc:  45.572917 [29936/37500]
loss:   0.933507, acc:  45.438034 [37424/37500]

Train Loss: 0.933576, Train Acc: 45.427%
Valid Loss: 0.929510, Valid Acc: 44.864%


Epoch 23
Learning Rate 0.000141
---------------------------------
loss:   0.930041, acc:  46.541132 [ 7472/37500]
loss:   0.929994, acc:  45.966880 [14960/37500]
loss:   0.930034, acc:  45.788818 [22448/37500]
loss:   0.932608, acc:  45.723157 [29936/37500]
loss:   0.933501, acc:  45

In [14]:
del model

best_model = NNModel().to(device)
best_model.load_state_dict(torch.load("../model/ka_s_model.pth"))

<All keys matched successfully>

In [15]:
test_loss, test_acc, test_pred = evaluate(test_dl, best_model, loss_fn, configs, return_pred=True)

print(f'Test Loss: {test_loss:>8f}, Test ACC: {test_acc:>8f}')

Test Loss: 0.939697, Test ACC: 0.484482
